In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time
import pandas as pd
import re
import moviepy

from lib import SQLiteConnection
from lib import get_info_video_judobase

In [2]:
# abrir Google Chrome
driver = webdriver.Chrome(r"C:\Users\manuel.lopez\Documents\chromedriver.exe")
try:
    driver.maximize_window()
except:
    pass

### SQL obtain all battles

category map

In [3]:
conn = SQLiteConnection('../judo.sql')

In [4]:
all_battles = conn.as_pandas('select * from battles', parse_dates=['date'], index_col='index').reset_index(drop=True)

In [5]:
competitors = conn.as_pandas('select * from competitors', index_col='index').reset_index(drop=True)

### Filter and extract some videos

problema: cuando falla uno salta una pestaña y jode el resto...

In [7]:
some_videos = all_battles[all_battles.category=='women_52'].url_video

for i, url_video in enumerate(some_videos):
    if i % 20 == 0:
        print(i)
    
    if url_video is None:
        continue
    # else
    
    try:
        local, opponent, url_youtube = get_info_video_judobase(driver, url_video)
    except:
        local, opponent, url_youtube = ('error',) * 3
        
    # assert 'https://www.youtube.com/watch?' in url_youtube, f'Strange url_youtube: {url_youtube}'

    conn.query(
        f'''
        INSERT OR IGNORE INTO videos_info 
        (url_video, url_youtube, local, opponent) 
        VALUES
        ("{url_video+'bb'}", "{url_youtube}", "{local}", "{opponent}");
        '''
    )

0
20
40
60
Retry...
80
100
120
140
160
180
200
220
240
260
280
300
320
340
360
380
400
420
440
460
480
500
520
540
560
580
600
620
640
660
680
700
720
740
760
780
800
820
840
860
880
900
920
940
960
980
1000
1020
1040
1060
1080
1100
1120
1140
1160
1180
1200
1220
1240
1260
1280
1300
1320
1340
1360
1380
1400
1420
1440
1460
1480
1500
1520
1540
1560
1580
1600
1620
1640
1660
1680
1700
1720
1740
1760
1780
1800
1820
1840
1860
1880
1900
1920
1940
1960
1980
2000
2020
2040
2060
2080
2100
2120
2140
2160
2180
2200
2220
2240
2260
2280
2300
2320
2340
2360
2380
2400
2420
2440
2460
2480
2500
2520
2540
2560
2580
2600
2620
2640
2660
2680
2700
2720
2740
2760
2780
2800
2820
2840
2860
2880
2900
2920
2940
2960
2980
3000
3020
3040
3060
3080
3100
3120
3140
3160
3180
3200
3220
3240
3260
3280
3300
3320
3340
3360
3380
3400
3420
3440
3460
3480


### Do all video_urls remaining

In [57]:
proposed_videos = all_battles.loc[all_battles.category == 'men_73', 'url_video']

In [58]:
proposed_videos

Series([], Name: url_video, dtype: object)

In [20]:
all_videos = conn.as_pandas('select * from videos_info')

In [33]:
for v in all_videos.url_video:
    print(v)

https://judobase.ijf.org/#/competition/contest/cont_open_por2018_w_0052_0031bb
https://judobase.ijf.org/#/competition/contest/gp_ned2018_w_0057_0032bb


In [32]:
all_battles.url_video.values.isin(all_videos.url_video.values).sum()

AttributeError: 'numpy.ndarray' object has no attribute 'isin'

In [29]:
all_videos.url_video.isin(all_battles.url_video)

0    False
1    False
Name: url_video, dtype: bool

In [24]:
all_battles.url_video.isin(all_videos.url_video).sum()

0

In [21]:
all_videos

,url_video,url_youtube,local,opponent
0,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=GoOQXMjmnaQ,GUICA Ecaterina,PLIIEVA Lyudmyla
1,https://judobase.ijf.org/#/competition/contest...,https://www.youtube.com/watch?v=3dDnTa1oD-0,BOROWSKA Anna,LU Tongjuan


### Extraer info y acceder al youtube

### Extraer info

In [41]:
# tabla con puntuaciones etiquetadas en tiempo
tabla_timed_points = driver.find_elements_by_tag_name('tbody')[1]

In [42]:
filas = tabla_timed_points.find_elements_by_tag_name('tr')

In [43]:
nrows = len(filas[0].find_elements_by_tag_name('td'))
nrows

3

In [45]:
if nrows == 2:
    for row_number, row in enumerate(filas):
        for col_number, value in enumerate(row.find_elements_by_tag_name('td')):
            print(row_number, col_number, value.text)

else:
    for row_number, row in enumerate(filas):
        for col_number, value in enumerate(row.find_elements_by_tag_name('td')):
            print(row_number, col_number, value.text)

    print(f'{nrows} rows!')

0 0 
0 1 00:41
0 2 Shido
Shido / Avoid-Grip
1 0 Shido
Shido / Non-Combativity
1 1 01:17
1 2 Shido
Shido / Non-Combativity
2 0 
2 1 01:54
2 2 Shido / Avoid-Grip
HSK (3rd shido)
3 rows!


In [26]:
seconds = 22

In [27]:
url_youtube_time = url_youtube + '&t=' + str(seconds)
url_youtube_time

'https://www.youtube.com/watch?v=TgFQMxD4aNs&t=22'

In [28]:
driver.get(url_youtube_time)

### Youtube downloads

In [29]:
from pytube import YouTube

In [30]:
y = YouTube(url_youtube_time)

In [31]:
[d for d in dir(y) if d[0]!='_']

['age_restricted',
 'caption_tracks',
 'captions',
 'description',
 'embed_html',
 'embed_url',
 'fmt_streams',
 'init',
 'initialize_caption_objects',
 'initialize_stream_objects',
 'js',
 'js_url',
 'length',
 'player_config_args',
 'prefetch',
 'prefetch_init',
 'rating',
 'register_on_complete_callback',
 'register_on_progress_callback',
 'stream_monostate',
 'streams',
 'thumbnail_url',
 'title',
 'vid_descr',
 'vid_info',
 'vid_info_url',
 'video_id',
 'views',
 'watch_html',
 'watch_url']

In [32]:
video = y.streams.filter(res='720p', mime_type='video/mp4').first()

In [33]:
assert video is not None, 'No video'

In [34]:
video.download(filename='judo720p3')

'C:\\Users\\manuel.lopez\\Documents\\scrape\\0e6a72df62d07aa6cc043c46d63ad5e7\\judo720p3.mp4'

### Edición con moviepy

In [1]:
3

3

In [3]:
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, concatenate_videoclips

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [5]:
# Load myHolidays.mp4 and select the subclip 00:00:50 - 00:00:60
clip = VideoFileClip('./my_concatenation.mp4')

In [9]:
clip.duration

15.05

In [14]:
corte = clip.subclip(4,8)

In [12]:
text0 = TextClip("Primer chunk", fontsize=70, color='red').set_position(('right', 'top'), relative=True).set_duration(2)
text1 = TextClip("Segundo chunk", fontsize=70, color='blue').set_position((0.4, 0), relative=True).set_duration(2)
# text2 = TextClip("Tercer chunk", fontsize=70, color='green').set_position((0.8, 0), relative=True).set_duration(5)

# textos = (text0, text1, text2)

In [13]:
textos = concatenate_videoclips([text0, text1])

In [15]:
final_clip = CompositeVideoClip([corte, textos])

In [16]:
final_clip.duration

4

In [17]:
final_clip.write_videofile("my_concatenatio2n2.mp4")

[MoviePy] >>>> Building video my_concatenatio2n2.mp4
[MoviePy] Writing audio in my_concatenatio2n2TEMP_MPY_wvf_snd.mp3


100%|█████████████████████████████████████████████████████████████████████████████████| 89/89 [00:00<00:00, 271.76it/s]


[MoviePy] Done.
[MoviePy] Writing video my_concatenatio2n2.mp4


 99%|███████████████████████████████████████████████████████████████████████████████▏| 100/101 [00:06<00:00, 12.44it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: my_concatenatio2n2.mp4 

